In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch 
from torch import nn 
from torch.utils.data import Dataset,DataLoader,TensorDataset, random_split 

import scipy.signal as signal
import scipy.stats as stats

In [2]:
##biovid dataset preparation
#read files
data = pd.read_csv("DATA/Biovid/input_gsr_part_a.csv", index_col = 0)
raw_signal_gsr = np.array(data.iloc[:,:-1])
biovid_label_gsr = np.array(data.iloc[:,-1])
print(raw_signal_gsr.shape)
print(biovid_label_gsr)

(8700, 2816)
[0 0 0 ... 4 4 4]


In [3]:
biovid_subject_id = np.repeat(range(87), 100)

In [4]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def personalization_apon(X, subject_ids):
    # 初始化 StandardScaler 对象
    scaler = StandardScaler()

    # 用于存储标准化后的特征
    standardized_features = []

    # 用于存储特征的索引，保持原始顺序
    feature_indices = []

    # 按照每个 subject id 进行标准化
    unique_subjects = np.unique(subject_ids)
    for subject in unique_subjects:
        # 找到当前主体的索引
        indices = np.where(subject_ids == subject)[0]
        # 获取当前主体的特征组
        feature_group = X.iloc[indices]

        # 进行标准化
        scaled_feature_group = scaler.fit_transform(feature_group)

        # 将标准化后的特征添加到列表
        standardized_features.extend(scaled_feature_group)
        # 将当前主体特征组的索引添加到列表
        feature_indices.extend(indices)

    # 将标准化后的特征转换为 DataFrame，注意要保持原始的列名和索引
    standardized_df = pd.DataFrame(standardized_features, columns=X.columns, index=feature_indices)
    return standardized_df

Biovid_gsr_standarlizad = personalization_apon(pd.DataFrame(raw_signal_gsr), biovid_subject_id)

In [5]:
Biovid_gsr_standarlizad

,0,1,2,3,4,5,6,7,8,9,...,2806,2807,2808,2809,2810,2811,2812,2813,2814,2815
0,3.559982,3.559512,3.555613,3.556382,3.553482,3.551429,3.551744,3.547813,3.548259,3.551689,...,0.154686,0.154291,0.157055,0.157379,0.152270,0.153768,0.160189,0.157484,0.154792,0.156852
1,-0.746816,-0.746189,-0.747061,-0.746263,-0.746025,-0.749018,-0.746759,-0.744122,-0.743265,-0.742950,...,-1.315966,-1.319208,-1.318225,-1.318040,-1.316100,-1.319685,-1.318533,-1.318655,-1.319953,-1.321111
2,-0.874570,-0.871681,-0.871596,-0.873341,-0.873162,-0.871579,-0.871954,-0.874162,-0.876367,-0.877079,...,-1.244782,-1.245045,-1.246619,-1.244232,-1.243662,-1.243630,-1.245730,-1.246489,-1.245105,-1.243866
3,1.081838,1.085052,1.085533,1.084637,1.083751,1.084825,1.083129,1.082674,1.082807,1.084188,...,-0.385657,-0.383373,-0.382114,-0.383860,-0.385957,-0.388122,-0.387590,-0.387378,-0.387440,-0.386252
4,0.109083,0.111159,0.105443,0.104107,0.106243,0.108498,0.110213,0.107043,0.108610,0.109168,...,-0.565196,-0.564936,-0.565482,-0.564843,-0.568487,-0.566584,-0.564971,-0.571146,-0.563996,-0.568474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8695,-0.772094,-0.772193,-0.772287,-0.772319,-0.772386,-0.772395,-0.772268,-0.771916,-0.771842,-0.771803,...,-0.694104,-0.694028,-0.694156,-0.694351,-0.694458,-0.694418,-0.694376,-0.694687,-0.694773,-0.694789
8696,0.029399,0.029437,0.029525,0.029515,0.029442,0.029535,0.029837,0.030168,0.030200,0.030243,...,-0.132102,-0.132141,-0.131308,-0.125532,-0.125715,-0.125815,-0.125835,-0.125994,-0.126078,-0.126212
8697,0.012346,0.012381,0.012465,0.011337,0.004969,0.011355,0.004238,0.004569,0.004603,0.004646,...,-0.402432,-0.402416,-0.402569,-0.402831,-0.402977,-0.403009,-0.402999,-0.403232,-0.403317,-0.403394
8698,-0.772094,-0.772193,-0.772287,-0.772319,-0.772386,-0.772395,-0.772268,-0.771916,-0.771842,-0.771803,...,-0.786585,-0.786491,-0.786610,-0.786784,-0.786879,-0.786816,-0.786764,-0.787100,-0.787186,-0.787183


In [6]:
##using EDA to train the model and use ecg as parameter weighting factor

import torch
from torch.utils.data import Dataset, DataLoader

# 自定义数据集类
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

# 自定义dataloader类
class MyDataLoader:
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        subject_data = {}
        for data in self.dataset:
            subject_id = data['subject_id']
            if subject_id not in subject_data:
                subject_data[subject_id] = []
            subject_data[subject_id].append(data)

        for subject_id, data_list in subject_data.items():
            num_batches = len(data_list) // self.batch_size
            for i in range(num_batches):
                batch_data = data_list[i*self.batch_size : (i+1)*self.batch_size]
                yield subject_id, batch_data

    def __len__(self):
        return len(self.dataset)

In [7]:
standarlized_eda_tensor = torch.tensor(Biovid_gsr_standarlizad.values, dtype=torch.float32)

In [8]:
standarlized_eda_tensor

tensor([[ 3.5600,  3.5595,  3.5556,  ...,  0.1575,  0.1548,  0.1569],
        [-0.7468, -0.7462, -0.7471,  ..., -1.3187, -1.3200, -1.3211],
        [-0.8746, -0.8717, -0.8716,  ..., -1.2465, -1.2451, -1.2439],
        ...,
        [ 0.0123,  0.0124,  0.0125,  ..., -0.4032, -0.4033, -0.4034],
        [-0.7721, -0.7722, -0.7723,  ..., -0.7871, -0.7872, -0.7872],
        [-0.7977, -0.7960, -0.7893,  ...,  0.4072,  0.4071,  0.4068]])

In [9]:
data = []
for i in range(standarlized_eda_tensor.shape[0]):
    sample = {
        'subject_id': biovid_subject_id[i],  # Assuming unique subject_ids starting from 0
        'signal': standarlized_eda_tensor[i],
        'label': biovid_label_gsr[i],
    }
    data.append(sample)

In [10]:
# 创建自定义数据集实例
ds_biovid = MyDataset(data)

In [12]:
def label_to_levels(label, num_classes, dtype=torch.float32):
    """Converts integer class label to extended binary label vector
    Parameters
    ----------
    label : int
        Class label to be converted into a extended
        binary vector. Should be smaller than num_classes-1.
    num_classes : int
        The number of class clabels in the dataset. Assumes
        class labels start at 0. Determines the size of the
        output vector.
    dtype : torch data type (default=torch.float32)
        Data type of the torch output vector for the
        extended binary labels.
    Returns
    ----------
    levels : torch.tensor, shape=(num_classes-1,)
        Extended binary label vector. Type is determined
        by the `dtype` parameter.
    Examples
    ----------
    >>> label_to_levels(0, num_classes=5)
    tensor([0., 0., 0., 0.])
    >>> label_to_levels(1, num_classes=5)
    tensor([1., 0., 0., 0.])
    >>> label_to_levels(3, num_classes=5)
    tensor([1., 1., 1., 0.])
    >>> label_to_levels(4, num_classes=5)
    tensor([1., 1., 1., 1.])
    """
    if not label <= num_classes-1:
        raise ValueError('Class label must be smaller or '
                         'equal to %d (num_classes-1). Got %d.'
                         % (num_classes-1, label))
    if isinstance(label, torch.Tensor):
        int_label = label.item()
    else:
        int_label = label

    levels = [1]*int_label + [0]*(num_classes - 1 - int_label)
    levels = torch.tensor(levels, dtype=dtype)
    return levels

def levels_from_labelbatch(labels, num_classes, dtype=torch.float32):
    """
    Converts a list of integer class label to extended binary label vectors
    Parameters
    ----------
    labels : list or 1D orch.tensor, shape=(num_labels,)
        A list or 1D torch.tensor with integer class labels
        to be converted into extended binary label vectors.
    num_classes : int
        The number of class clabels in the dataset. Assumes
        class labels start at 0. Determines the size of the
        output vector.
    dtype : torch data type (default=torch.float32)
        Data type of the torch output vector for the
        extended binary labels.
    Returns
    ----------
    levels : torch.tensor, shape=(num_labels, num_classes-1)
    Examples
    ----------
    >>> levels_from_labelbatch(labels=[2, 1, 4], num_classes=5)
    tensor([[1., 1., 0., 0.],
            [1., 0., 0., 0.],
            [1., 1., 1., 1.]])
    """
    levels = []
    for label in labels:
        levels_from_label = label_to_levels(
            label=label, num_classes=num_classes, dtype=dtype)
        levels.append(levels_from_label)

    levels = torch.stack(levels)
    return levels

def proba_to_label(probas):
    """
    Converts predicted probabilities from extended binary format
    to integer class labels
    Parameters
    ----------
    probas : torch.tensor, shape(n_examples, n_labels)
        Torch tensor consisting of probabilities returned by CORAL model.
    Examples
    ----------
    >>> # 3 training examples, 6 classes
    >>> probas = torch.tensor([[0.934, 0.861, 0.323, 0.492, 0.295],
    ...                        [0.496, 0.485, 0.267, 0.124, 0.058],
    ...                        [0.985, 0.967, 0.920, 0.819, 0.506]])
    >>> proba_to_label(probas)
    tensor([2, 0, 5])
    """
    predict_levels = probas > 0.5
    predicted_labels = torch.sum(predict_levels, dim=1)
    return predicted_labels

def coral_loss(logits, levels, importance_weights=None, reduction='mean'):
    """Computes the CORAL loss described in
    Cao, Mirjalili, and Raschka (2020)
    *Rank Consistent Ordinal Regression for Neural Networks
       with Application to Age Estimation*
    Pattern Recognition Letters, https://doi.org/10.1016/j.patrec.2020.11.008
    Parameters
    ----------
    logits : torch.tensor, shape(num_examples, num_classes-1)
        Outputs of the CORAL layer.
    levels : torch.tensor, shape(num_examples, num_classes-1)
        True labels represented as extended binary vectors
        (via `coral_pytorch.dataset.levels_from_labelbatch`).
    importance_weights : torch.tensor, shape=(num_classes-1,) (default=None)
        Optional weights for the different labels in levels.
        A tensor of ones, i.e.,
        `torch.ones(num_classes-1, dtype=torch.float32)`
        will result in uniform weights that have the same effect as None.
    reduction : str or None (default='mean')
        If 'mean' or 'sum', returns the averaged or summed loss value across
        all data points (rows) in logits. If None, returns a vector of
        shape (num_examples,)
    Returns
    ----------
        loss : torch.tensor
        A torch.tensor containing a single loss value (if `reduction='mean'` or '`sum'`)
        or a loss value for each data record (if `reduction=None`).
    Examples
    ----------
    >>> import torch
    >>> from coral_pytorch.losses import coral_loss
    >>> levels = torch.tensor(
    ...    [[1., 1., 0., 0.],
    ...     [1., 0., 0., 0.],
    ...    [1., 1., 1., 1.]])
    >>> logits = torch.tensor(
    ...    [[2.1, 1.8, -2.1, -1.8],
    ...     [1.9, -1., -1.5, -1.3],
    ...     [1.9, 1.8, 1.7, 1.6]])
    >>> coral_loss(logits, levels)
    tensor(0.6920)
    """

    if not logits.shape == levels.shape:
        raise ValueError("Please ensure that logits (%s) has the same shape as levels (%s). "
                         % (logits.shape, levels.shape))

    term1 = (F.logsigmoid(logits)*levels
                      + (F.logsigmoid(logits) - logits)*(1-levels))

    if importance_weights is not None:
        term1 *= importance_weights

    val = (-torch.sum(term1, dim=1))

    if reduction == 'mean':
        loss = torch.mean(val)
    elif reduction == 'sum':
        loss = torch.sum(val)
    elif reduction is None:
        loss = val
    else:
        s = ('Invalid value for `reduction`. Should be "mean", '
             '"sum", or None. Got %s' % reduction)
        raise ValueError(s)

    return loss

import torch.optim.lr_scheduler as lr_scheduler
#two self_defined class
class CoralLayer(torch.nn.Module):
        """ 
        -----------
        size_in : int
            Number of input features for the inputs to the forward method, which
            are expected to have shape=(num_examples, num_features).
        num_classes : int
            Number of classes in the dataset.
        preinit_bias : bool (default=True)
            If true, it will pre-initialize the biases to descending values in
            [0, 1] range instead of initializing it to all zeros. This pre-
            initialization scheme results in faster learning and better
            generalization performance in practice.
        """
        def __init__(self, size_in, num_classes, preinit_bias=True):
            super().__init__()
            self.size_in, self.size_out = size_in, 1

            self.coral_weights = torch.nn.Linear(self.size_in, 1, bias=False)
            if preinit_bias:
                self.coral_bias = torch.nn.Parameter(
                    torch.arange(num_classes - 1, 0, -1).float() / (num_classes-1))
            else:
                self.coral_bias = torch.nn.Parameter(
                    torch.zeros(num_classes-1).float())

        def forward(self, x):
            """
            Computes forward pass.
            Parameters
            -----------
            x : torch.tensor, shape=(num_examples, num_features)
                Input features.
            Returns
            -----------
            logits : torch.tensor, shape=(num_examples, num_classes-1)
            """
            return self.coral_weights(x) + self.coral_bias

In [13]:
class ConvNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        self.features = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels = 1, out_channels = 3,kernel_size = 3, stride = 1, padding = 1),
            torch.nn.BatchNorm1d(3),
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.AvgPool1d(kernel_size=2, stride=2),
            torch.nn.Conv1d(in_channels = 3, out_channels = 6, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.BatchNorm1d(6),
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.AvgPool1d(kernel_size=2, stride=2))

        ### Specify CORAL layer
        self.fc = CoralLayer(size_in=4224, num_classes=num_classes)
        #self.fc = nn.Linear(4224, num_classes - 1)
        ###--------------------------------------------------------------------###

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1) # flatten

        ##### Use CORAL layer #####
        logits =  self.fc(x)
        probas = torch.sigmoid(logits)
        ###--------------------------------------------------------------------###

        return logits, probas

In [20]:
import torch.optim as optim
def new_coral_training(dl_train, num_classes, n_epochs=20):
    classifier = ConvNet(num_classes) 
    optimizer = optim.Adam(classifier.parameters())

    for epoch in range(n_epochs):
        classifier = classifier.train()
        for subject_id, batch_data in dl_train:
            inputs = [item["signal"] for item in batch_data]
            labels = [item["label"] for item in batch_data]

            inputs = torch.stack(inputs, dim=0)
            labels = torch.tensor(labels)
            
            inputs = inputs.unsqueeze(1)
            # Prediction on source data
            logits, probas = classifier(inputs)
            
            # Convert class labels for CORAL
            levels = levels_from_labelbatch(labels, num_classes=5)
            
            # Compute loss
            regression_loss = coral_loss(logits, levels)
            
            optimizer.zero_grad()
            regression_loss.backward()
            optimizer.step()

            #print(f"Epoch [{epoch}/{n_epochs}], Regression Loss: {regression_loss.item():.4f}")
        
    return classifier

In [25]:
def new_compute_mae_and_mse(model, data_loader):
    # Set the model to evaluation mode
    with torch.no_grad():
        mae, mse, acc, num_examples = 0., 0., 0., 0

        for subject_id, batch_data in data_loader:
            inputs = [item["signal"] for item in batch_data]
            labels = [item["label"] for item in batch_data]

            inputs = torch.stack(inputs, dim=0)
            labels = torch.tensor(labels)
            
            inputs = inputs.unsqueeze(1)
            # Prediction on source data
            logits, probas = model(inputs)
            
            predicted_labels = proba_to_label(probas).float()
            
            print(predicted_labels)
            print(labels)
            
            num_examples += labels.size(0)

            mae += torch.sum(torch.abs(predicted_labels - labels))
            mse += torch.sum((predicted_labels - labels)**2)
            acc += torch.sum(predicted_labels == labels).item()

        mae = mae / num_examples
        mse = mse / num_examples
        acc = acc / num_examples
    return mae, mse, acc
        

In [22]:
from torch.utils.data import Subset
import torch.nn.functional as F

def LOSO():
    total_valid_file = [i for i in range(87)]
    subject_id = torch.tensor(biovid_subject_id)

    total_mae = []
    total_mse = []

    for subject in total_valid_file:
        print(subject)
        test_mask = (subject_id == subject)
        training_mask = (subject_id != subject)

        heldout_data = Subset(ds_biovid, np.where(test_mask)[0])
        training_data = Subset(ds_biovid, np.where(training_mask)[0])

        # 使用DataLoader加载数据集
        dl_train = MyDataLoader(training_data,batch_size = 32)
        dl_val = MyDataLoader(heldout_data,batch_size = 32)
        
        classifier = new_coral_training(dl_train, num_classes = 5, n_epochs=20)
        
        # Evaluate target data
        test_mae, test_mse = new_compute_mae_and_mse(classifier, dl_val)
        total_mae.append(test_mae)
        total_mse.append(test_mse)
        
    print(len(total_mae))
    print(np.mean(total_mae))
    print(np.mean(total_mse))
    return total_mae, total_mse

In [23]:
total_mae, total_mse = LOSO()

0
tensor([0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 3., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 3., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([3., 3., 2., 4., 3., 4., 3., 2., 3., 3., 3., 3., 2., 3., 2., 3., 4., 3.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 3., 4., 3.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
1
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 

tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 2., 2., 1., 3., 4., 3., 2., 2., 0., 1., 1., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 3., 2., 3., 3., 2., 1., 2., 3., 0., 2., 2., 1., 2., 2., 2., 3., 2.,
        2., 2., 3., 2., 3., 3., 1., 3., 3., 3., 0., 3., 3., 3.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([3., 3., 4., 2., 3., 3., 4., 3., 4., 4., 3., 4., 4., 4., 3., 2., 0., 3.,
        4., 1., 4., 3., 1., 4., 3., 3., 4., 2., 3., 4., 3., 4.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
12
tensor([1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 2.,
        2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 4., 1., 2., 1., 3., 1., 1., 1., 1., 1., 1., 3., 2., 2., 2., 2., 2.,
        1., 1., 2., 2., 1., 1., 0., 2., 1., 2., 2., 1., 3., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([2., 2., 2., 2., 3., 3., 2., 3., 3., 3., 3., 4., 3., 2., 2., 2., 3., 3.,
        3., 3., 3., 2., 2., 2., 2., 3., 3., 3., 3., 4., 4., 4.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
23
tensor([0., 3., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1.,
        1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 3., 2., 2., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 2., 2., 2., 2., 3., 3.,
        2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
34
tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([3., 1., 1., 2., 4., 3., 3., 3., 1., 2., 2., 3., 3., 3., 4., 1., 3., 2.,
        3., 3., 3., 2., 2., 2., 1., 2., 3., 1., 1., 1., 1., 3.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([2., 3., 4., 3., 3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4.,
        3., 3., 2., 3., 3., 3., 3., 4., 3., 3., 2., 3., 3., 3.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
45
tensor([1., 1., 1., 2., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 2., 1.,
        0., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 2.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([1., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 1., 2., 2., 2.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 3., 2.,
        4., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
56
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 0., 2., 1., 1., 2., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 2., 2., 2., 1., 1., 2., 2., 2.,
        2., 1., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 2.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 1., 1., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 1., 1., 2., 2., 2.,
        2., 2., 1., 1., 1., 1., 2., 2., 2., 1., 2., 2., 2., 1.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([1., 1., 2., 1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2.,
        2., 2., 1., 1., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
67
tensor([1., 2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 1., 2.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 2., 1., 0., 1., 2., 1., 1.,
        1., 1., 2., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1., 2., 2., 1., 2., 2., 1., 2., 2., 2., 1., 1., 1., 1., 1., 0., 1., 0.,
        2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 4., 3.,
        2., 2., 3., 3., 2., 1., 2., 2., 2., 2., 3., 2., 2., 2.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
78
tensor([0., 0., 0., 1., 1., 0., 2., 1., 0., 0., 0., 3., 1., 0., 0., 0., 1., 1.,
        3., 1., 0., 1., 0., 0., 0., 1., 1., 1., 3., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

In [26]:
from torch.utils.data import Subset
import torch.nn.functional as F

def LOSO_with_acc():
    total_valid_file = [i for i in range(87)]
    subject_id = torch.tensor(biovid_subject_id)

    total_mae = []
    total_mse = []
    total_acc = []

    for subject in total_valid_file:
        print(subject)
        test_mask = (subject_id == subject)
        training_mask = (subject_id != subject)

        heldout_data = Subset(ds_biovid, np.where(test_mask)[0])
        training_data = Subset(ds_biovid, np.where(training_mask)[0])

        # 使用DataLoader加载数据集
        dl_train = MyDataLoader(training_data,batch_size = 32)
        dl_val = MyDataLoader(heldout_data,batch_size = 32)
        
        classifier = new_coral_training(dl_train, num_classes = 5, n_epochs=20)
        
        # Evaluate target data
        test_mae, test_mse, test_acc = new_compute_mae_and_mse(classifier, dl_val)
        total_mae.append(test_mae)
        total_mse.append(test_mse)
        total_acc.append(test_acc)
        
    print(len(total_mae))
    print(np.mean(total_mae))
    print(np.mean(total_mse))
    print(np.mean(total_acc))
    return total_mae, total_mse, total_acc

In [27]:
total_mae, total_mse, total_acc = LOSO_with_acc()

0
tensor([1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
        1., 1., 2., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2., 3., 2., 0., 1., 1., 1., 1.,
        0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 2., 3., 3., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([3., 3., 2., 3., 3., 3., 2., 2., 3., 2., 3., 2., 3., 2., 2., 3., 3., 3.,
        4., 4., 4., 3., 4., 4., 4., 3., 3., 3., 3., 3., 4., 3.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
1
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 

tensor([1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 2., 2., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([3., 3., 2., 3., 2., 3., 1., 3., 3., 1., 2., 2., 1., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([4., 4., 4., 3., 4., 3., 4., 3., 3., 4., 3., 3., 4., 4., 3., 2., 1., 3.,
        4., 2., 4., 4., 2., 4., 3., 3., 3., 3., 3., 4., 4., 4.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
12
tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 2., 2., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([2., 0., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 2., 0., 1., 1., 1., 2.,
        2., 1., 4., 3., 1., 1., 1., 1., 2., 2., 1., 1., 2., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 4., 1., 2., 1., 4., 1., 2., 1., 1., 1., 1., 3., 2., 2., 2., 2., 3.,
        2., 2., 2., 2., 1., 2., 0., 2., 2., 2., 2., 1., 3., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([3., 2., 2., 2., 3., 2., 3., 3., 3., 3., 3., 4., 3., 2., 2., 1., 4., 3.,
        3., 3., 4., 2., 2., 2., 2., 3., 3., 3., 3., 4., 4., 4.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
23
tensor([1., 2., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
        1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([1., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([2., 2., 2., 2., 2., 3., 2., 2., 1., 1., 1., 1., 2., 1., 1., 2., 3., 2.,
        2., 2., 2., 3., 3., 2., 3., 2., 2., 2., 2., 2., 1., 2.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
34
tensor([2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([1., 1., 2., 3., 4., 3., 2., 4., 2., 3., 3., 2., 1., 2., 4., 1., 2., 3.,
        2., 1., 2., 2., 3., 3., 2., 2., 3., 3., 1., 3., 2., 3.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 2., 1., 0., 1., 0., 1., 1., 2., 2., 1., 2., 1., 1., 2., 1., 1., 1.,
        1., 0., 1., 1., 1., 2., 1., 1., 2., 2., 3., 2., 2., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([3., 3., 4., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 1., 1., 2., 4.,
        4., 3., 3., 2., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
45
tensor([1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 2., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 1., 2., 1., 1., 1.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 2., 1., 1., 1., 1., 1., 1., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 1., 1., 2., 2., 1., 2., 2., 2., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 2., 1., 2., 2., 3., 3.,
        3., 3., 2., 2., 2., 3., 2., 2., 2., 1., 2., 2., 2., 2.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
56
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 3., 1., 1., 2., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 3., 2., 2., 1., 1., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 2.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([2., 1., 1., 2., 3., 3., 1., 2., 2., 2., 2., 1., 1., 1., 1., 2., 2., 2.,
        2., 1., 1., 1., 1., 1., 2., 2., 2., 1., 2., 2., 1., 1.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([1., 1., 2., 2., 2., 1., 1., 0., 0., 0., 0., 1., 2., 3., 2., 2., 3., 2.,
        2., 2., 2., 1., 1., 1., 2., 2., 2., 2., 2., 2., 1., 2.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
67
tensor([2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1

tensor([0., 1., 2., 2., 2., 0., 0., 1., 1., 1., 2., 2., 1., 0., 2., 1., 1., 2.,
        2., 2., 3., 1., 2., 2., 2., 0., 0., 0., 1., 2., 2., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([0., 2., 1., 1., 2., 2., 1., 2., 2., 3., 2., 2., 2., 2., 2., 1., 0., 1.,
        0., 1., 0., 1., 1., 1., 2., 2., 2., 2., 2., 3., 2., 2.])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3])
tensor([3., 2., 2., 2., 1., 2., 3., 2., 2., 2., 1., 2., 3., 2., 2., 3., 4., 4.,
        3., 4., 4., 4., 3., 2., 3., 3., 1., 2., 3., 2., 2., 3.])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])
78
tensor([0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 3., 1., 0., 0., 0., 1., 1.,
        3., 1., 0., 1., 0., 1., 0., 1., 1., 1., 4., 1., 1., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1